### Conectar a Neo4j Desktop

In [1]:
import pandas as pd
from neo4j import GraphDatabase
import time

In [2]:
df = pd.read_csv('C:/Users/migue/AplicacionesYTendencias/ProyectoAyT/UEM-Analytics-G22-3/BBDD-Grafos/data_advantege actor critic2.csv')
df.head()

,titulo,autores,abstract,clase_pri,clase_otr,id
0,Reinforcement Learning through Asynchronous Ad...,Mohammad Babaeizadeh,We introduce a hybrid CPU/GPU version of the...,Machine Learning (cs.LG),NaN,0
1,Altruistic Maneuver Planning for Cooperative A...,Behrad Toghi,With the adoption of autonomous vehicles on ...,Robotics (cs.RO),NaN,1
2,Towards Understanding Asynchronous Advantage A...,Han Shen,Asynchronous and parallel implementation of ...,Machine Learning (cs.LG),; Optimization and Control (math.OC),2
3,The Advantage Regret-Matching Actor-Critic,Audrūnas Gruslys,Regret minimization has played a key role in...,Artificial Intelligence (cs.AI),; Machine Learning (cs.LG),3
4,Actor-Critic Sequence Training for Image Capti...,Li Zhang,Generating natural language descriptions of ...,Computer Vision and Pattern Recognition (cs.CV),NaN,4


In [3]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [4]:
conn = Neo4jConnection(uri="bolt://localhost:7687", 
                       user="usermate",              
                       pwd="glass")

### Creación BBDD

In [5]:
conn.query('CREATE CONSTRAINT papers IF NOT EXISTS FOR (p:Paper) REQUIRE p.id IS UNIQUE')
conn.query('CREATE CONSTRAINT authors IF NOT EXISTS FOR (a:Author) REQUIRE a.name IS UNIQUE')
conn.query('CREATE CONSTRAINT categories IF NOT EXISTS FOR (c:Category) REQUIRE c.category IS UNIQUE')

[]

In [17]:
def add_categories(categories):
    # Añade a la BBDD de grafos de Neo4j el nodo de categorías.
    query = '''
            UNWIND $rows AS row
            MERGE (c:Category {category: row.category})
            RETURN count(*) as total
            '''
        
    return conn.query(query, {'rows':categories})
    #return conn.query(query, parameters = {'rows':categories.to_dict('records')})


In [7]:
def add_authors(rows, batch_size=10000):
    # Añade a la BBDD de grafos de Neo4j el nodo de autores.
    # Adds author nodes to the Neo4j graph as a batch job.
    query = '''
            UNWIND $rows AS row
            MERGE (:Author {name: row.author})
            RETURN count(*) as total
            '''
    return insert_data(query, rows, batch_size)

def insert_data(query, rows, batch_size = 10000):
    # Funcición para gestionar las cargas de la BBDD de Neo4j en modo batch.
    # Puede ser útil para cuando se tengan más de 50.000 cargas.
    
    total = 0
    batch = 0
    start = time.time()
    result = None
    
    while batch * batch_size < len(rows):

        res = conn.query(query, 
                         parameters = {'rows': rows[batch*batch_size:(batch+1)*batch_size].to_dict('records')})
        total += res[0]['total']
        batch += 1
        result = {"total":total, 
                  "batches":batch, 
                  "time":time.time()-start}
        print(result)
        
    return result

In [8]:
def add_papers(rows, batch_size=5000):
   # Se Añade a la BBDD de grafos de Neo4j el nodo de los papers.

   # Añade el nodo de papers (:Author)--(:Paper)  
   # (:Paper)--(:Category) y las relaciones a la BBDD de grafos de Neo4j en modo batch.
 
   query = '''
   UNWIND $rows as row
   MERGE (p:Paper {id:row.id}) ON CREATE SET p.title = row.title
 
   // connect categories
   WITH row, p
   UNWIND row.clase_pri AS category_name
   MATCH (c:Category {category: category_name})
   MERGE (p)-[:IN_CATEGORY]->(c)
 
   // connect authors
   WITH distinct row, p // reduce cardinality
   UNWIND row.autores AS author
   MATCH (a:Author {name: author})
   MERGE (a)-[:AUTHORED]->(p)
   RETURN count(distinct p) as total
   '''
 
   return insert_data(query, rows, batch_size)

In [9]:
df.head()

,titulo,autores,abstract,clase_pri,clase_otr,id
0,Reinforcement Learning through Asynchronous Ad...,Mohammad Babaeizadeh,We introduce a hybrid CPU/GPU version of the...,Machine Learning (cs.LG),NaN,0
1,Altruistic Maneuver Planning for Cooperative A...,Behrad Toghi,With the adoption of autonomous vehicles on ...,Robotics (cs.RO),NaN,1
2,Towards Understanding Asynchronous Advantage A...,Han Shen,Asynchronous and parallel implementation of ...,Machine Learning (cs.LG),; Optimization and Control (math.OC),2
3,The Advantage Regret-Matching Actor-Critic,Audrūnas Gruslys,Regret minimization has played a key role in...,Artificial Intelligence (cs.AI),; Machine Learning (cs.LG),3
4,Actor-Critic Sequence Training for Image Capti...,Li Zhang,Generating natural language descriptions of ...,Computer Vision and Pattern Recognition (cs.CV),NaN,4


In [15]:
categories = pd.DataFrame(df[['clase_pri']])
categories.rename(columns={'clase_pri':'category'},
                  inplace=True)
categories = categories.explode('category') \
                       .drop_duplicates(subset=['category'])

authors = pd.DataFrame(df[['autores']])
authors.rename(columns={'autores':'author'},
               inplace=True)
authors=authors.explode('author').drop_duplicates(subset=['author'])

In [18]:
add_categories(categories)

Query failed: Values of type <class 'pandas.core.frame.DataFrame'> are not supported


In [11]:
add_authors(authors)

TypeError: query() got an unexpected keyword argument 'parameters'

In [12]:
add_papers(df)

TypeError: query() got an unexpected keyword argument 'parameters'